In [1]:
pip install arcgis folium requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import folium

In [3]:
# Establish a connection to your GIS.
from arcgis.gis import GIS
from IPython.display import display
gis = GIS() # anonymous connection to www.arcgis.com

In [4]:
criticalinfra = gis.content.get('3d92487b911f4c3ab8322a8ead078b6e')

In [5]:
print("ArcGIS Item Title:", criticalinfra.title)

ArcGIS Item Title: WVDHSEM Critical Infrastructure Map


In [6]:
infra_layer = criticalinfra.layers[0]

In [7]:
for layer in criticalinfra.layers:
    print(layer.properties.name)

911 Centers
Correctional Facilities
EMS Stations 
Fire Departments
Health Departments
Higher Education Locations
Hospitals
Nursing Homes
OES Centers
Police Departments
Primary Care Centers
Private Schools
Public Libraries
Public Schools
Counties


In [8]:
# Step 4: Query the layer to fetch spatial data
query_result = infra_layer.query(where="1=1", out_fields="*", return_geometry=True)
features = query_result.features

# Debug: Check the number of features and their geometries
print(f"Number of features: {len(features)}")
for feature in features[:5]:  # Limit output for brevity
    print(feature.geometry)

# Extract coordinates for mapping
coordinates = []
for feature in features:
    geom = feature.geometry
    if geom and 'x' in geom and 'y' in geom:
        coordinates.append((geom['y'], geom['x']))  # (latitude, longitude)

# Debug: Check extracted coordinates
print("Extracted coordinates:", coordinates[:5])  # Print first 5 for brevity

# Step 5: Create a Folium map and plot critical infrastructure
map_center = [38.5976, -80.4549]  # Centering the map around WV
map = folium.Map(location=map_center, zoom_start=7)

# Add points to the map
if coordinates:
    for coord in coordinates:
        folium.Marker(location=coord, popup="Critical Infrastructure").add_to(map)
else:
    print("No coordinates were extracted.")

# Display the map
map

Number of features: 54
{'x': 586123.4405430289, 'y': 4331382.061512335, 'spatialReference': {'wkid': 26917, 'latestWkid': 26917}}
{'x': 761180.7538408719, 'y': 4370775.616701526, 'spatialReference': {'wkid': 26917, 'latestWkid': 26917}}
{'x': 426735.30791592994, 'y': 4215370.516935042, 'spatialReference': {'wkid': 26917, 'latestWkid': 26917}}
{'x': 529820.5903207599, 'y': 4283109.960733527, 'spatialReference': {'wkid': 26917, 'latestWkid': 26917}}
{'x': 532848.8175746963, 'y': 4457760.0443993965, 'spatialReference': {'wkid': 26917, 'latestWkid': 26917}}
Extracted coordinates: [(4331382.061512335, 586123.4405430289), (4370775.616701526, 761180.7538408719), (4215370.516935042, 426735.30791592994), (4283109.960733527, 529820.5903207599), (4457760.0443993965, 532848.8175746963)]


In [9]:
feature_service_url = "https://services3.arcgis.com/XQfPLTVDElDOMEad/arcgis/rest/services/WVDHSEM_Critical_Infrastructure_Map/FeatureServer/0/query"

params = {
    "where": "1=1",  # Fetch all data
    "outFields": "*",  # Get all fields
    "returnGeometry": "true",
    "f": "geojson"  # Request GeoJSON format
}

response = requests.get(feature_service_url, params=params)

geojson_data = response.json()
#print(geojson_data)

if response.status_code != 200:
    raise Exception(f"Failed to fetch data from the FeatureServer: {response.text}")

for feature in geojson_data['features']:
    properties = feature['properties']
    properties['FACILITY'] = properties.get('FACILITY', 'Unknown Facility')
    properties['ADDRESS'] = properties.get('ADDRESS', 'Unknown Address')
    properties['CITY'] = properties.get('CITY', 'Unknown City')
    
geojson_data = response.json()

# Step 3: Add the new layer to the existing Folium map
new_layer = folium.GeoJson(
    geojson_data,
    name="Critical Infrastructure (FeatureServer)",
    popup=folium.GeoJsonPopup(
        fields=['FACILITY', 'ADDRESS', 'CITY'],  # Valid fields
        labels=True
    )
)

new_layer.add_to(map)

# Step 4: Add a layer control to toggle layers on the map
folium.LayerControl().add_to(map)

# Display the updated map
map